In [1]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 999
import numpy as np

from ipywidgets import Button, HBox, VBox, Layout, interactive_output, GridspecLayout, AppLayout
from IPython.display import display

### Game data entry
The code chunk below includes helper functions to retrieve roster data for any Section V team (well, at least for most as of right now). Run the code to get an empty dataframe for Fairport *and/or* our opponent to store game data.<br>

Button icons can be found here: https://fontawesome.com/v4.7.0/icons/

In [2]:
# Returns list of names of icons
def icons():
    icons = ['bandcamp', 'bath', 'window-close', 'user-o', 'podcast', 'user-circle-o', 'adjust', 'bank', 'bolt',
             'bullhorn', 'gavel', 'filter', 'meh-o', 'plug', 'snowflake-o', 'toggle-on', 'wifi', 'pinterest',
             'google-plus', 'black-tie', 'envira', 'chrome', 'delicious', 'btc', 'usb', 'linux', 'wikipedia-w',
             'windows', 'yelp', 'youtube', 'apple', 'pause', 'play', 'hand-o-up']
    return icons

### Advanced Stats

Now that we're prepared a dataframe to store all of the data you'll be keeping track of, we have to define some function to calculate advanced stats that are based off of the values you're collecting.

In [3]:
class GameData:
    
    def __init__(self, team):
        self.team = team # Store team name
        self.stats = self.make_stats_dict()
        self.data, self.n = self.prep_game_df() # Get empty roster and longest str length
        self.stats_update_dict = self.make_stats_update_dict()
        
    # This function organizes stats we're recording for data entry into a dict to organize update functionality
    def make_stats_update_dict(self):
        stats_dict = {
            '2PA': self.update_FGA, '3PA': self.update_FGA,
            '2PM': self.update_FGM, '3PM': self.update_FGM,
            'FTA': self.update_FTA, 'FTM': self.update_FTM,
            'OREB': self.update_REB, 'DREB': self.update_REB,
            'AST': self.update_AST_or_TO, 'TOV': self.update_AST_or_TO,
            'STL': self.update_defensive, 'DFL': self.update_defensive, 
            'BLK': self.update_defensive, 'PFL': self.update_defensive,
        }
        return stats_dict
        
    # Returns list of stats
    def make_stats_dict(kind='entry'):
        stats = {
            'entry': ['2PM', '2PA', '3PM', '3PA', 'FTM', 'FTA', 'DREB', 'OREB', 'AST', 
                      'STL', 'DFL', 'BLK', 'TOV', 'PFL', 'CHARGE', 'PENNY'],
            'totals': ['PTS', 'FGM', 'FGA', 'FG%', 'FT%', '2P%', '3P%', 'REB', 'HUSTLE'],
            'advanced': ['eFG%', 'TS%', 'AST/TO'],
            'misc': ['PACE', 'OFFRTG', 'DEFRTG', 'NETRTG', 'USG%', 'PACE', 'AST Ratio', 'OREB%', 'DREB%', 'PAINT TOUCH']
        }
        stats['all'] = stats['totals'] + stats['entry'] + stats['advanced'] + stats['misc']
        return stats
        
    # Return df to fill with roster data for data entry
    # Available rosters: ['Fairport', Sutherland', 'Mendon', 'Schroeder', 'Gates', 'Hilton', 'Victor', 'McQ', 'BK', 'RH']
    def get_roster(self, n_players=18):
        print("Retrieving {} roster data.".format(self.team))
        while n_players >= 5:
            try:
                df = pd.read_excel('fairport_data/rosters.xlsx', sheet_name=self.team, header=2, nrows=n_players, 
                                   usecols=[0,1], dtype={'#': int, "Name": str})
                return df
            except:
                n_players -= 1
        print('Unsuccessful - please try again.')
        return None
        
    # Create empty df to store data entries
    def prep_game_df(self):
        roster = self.get_roster() # Retrieve roster data
        roster["Player ID"] = "#" + roster["#"].astype(str) + " " + roster["Name"]
        for stat in self.stats['all']:
            roster[stat] = 0 # Add stats as col to dataframe
        len_longest_id = roster["Player ID"].apply(lambda x: len(x)).max() + 2
        roster.set_index("Player ID", inplace=True)
        return roster.iloc[:,2:], len_longest_id
    
    """The functionality below handles automatic updates to game data upon entry"""
    
    # Perform update for: True Shooting % = PTS/[2*(FGA + 0.44*FTA)]
    def update_ts(self, name):
        self.data.loc[name, 'TS%'] = self.data.loc[name,'PTS']/(2*(self.data.loc[name,'FGA'] + 0.44*self.data.loc[name,'FTA']))
    
    # Perform update for: Effective Field Goal % = ((FGM + (0.5)*3PM))/FGA
    def update_efg(self, name):
        self.data.loc[name, 'eFG%'] = (self.data.loc[name,'FGM'] + 0.5*self.data.loc[name,'3PM'])/self.data.loc[name,'FGA']
    
    # Perform update for player free throw percentage
    def update_ft_pct(self, name):
        self.data.loc[name,'FT%'] = self.data.loc[name,'FTM']/self.data.loc[name,'FTA']
    
    # Perform update for AST/TO ratio
    def update_ASTTO(self, name):
        self.data.loc[name,'AST/TO'] = self.data.loc[name,'AST']/self.data.loc[name,'TOV']
    
    # Update splits impacted by a field goal make: FG%, eFG%, TS%
    def update_general_splits(self, name):
        self.data.loc[name,'FG%'] = self.data.loc[name,'FGM']/self.data.loc[name,'FGA'] # Update FG%
        self.update_ts(name) # Update TS%
        self.update_efg(name) # Update eFG%
        
    # Performs update for any field goal attempt (i.e. 2PA, 3PA)
    def update_FGA(self, name, stat):
        self.data.loc[name, [stat,'FGA']] += 1 # Increment 2/3PA + FGA
        self.data.loc[name,stat[:-1]+'%'] = self.data.loc[name,stat[:-1]+'M']/self.data.loc[name,stat] # Update 2/3P%
        self.update_general_splits(name) # Update FG%, eFG%, TS%
    
    # Performs update for any field goal make (i.e. 2PM, 3PM)
    def update_FGM(self, name, stat):
        #stats_list = [stat, stat[:-1]+'A', 'FGA', 'FGM']
        self.data.loc[name, [stat, stat[:-1]+'A', 'FGA', 'FGM']] += 1 # Increment 2/3PA, 2/3PM and FGA/FGM
        self.data.loc[name, stat[:-1]+'%'] = self.data.loc[name, stat]/self.data.loc[name, stat[:-1]+'A'] # Update 2/3P%
        self.data.loc[name,'PTS'] += int(stat[0]) # Increment point total
        self.update_general_splits(name) # Update FG%, eFG%, TS%        
    
    # Performs update for a free throw atempt
    def update_FTA(self, name, stat):
        self.data.loc[name,stat] += 1 # Increment FTA
        self.update_ft_pct(name) # Update FT%
        self.update_ts(name) # Update TS%
    
    # Performs update for a free throw make
    def update_FTM(self, name, stat):
        stats_list = [stat, stat[:-1]+'A','PTS']
        self.data.loc[name, stats_list] += 1 # Increment FTA, FTM, PTS
        self.update_ft_pct(name) # Update FT%
        self.update_ts(name) # Update TS%
        
    # Performs update for a rebound
    def update_REB(self, name, stat):
        self.data.loc[name, [stat, 'REB']] += 1 # Increment O/DREB, REB
        # Add functionality to account for number of possessions
    
    # Performs update for the following defensive stats: steals, deflections, blocks, personal fouls
    def update_defensive(self, name, stat):
        self.data.loc[name, stat] += 1
    
    # Performs update for assist or turnover
    def update_AST_or_TO(self, name, stat):
        self.data.loc[name, stat] += 1 # Increment AST/TO
        self.update_ASTTO(name) # Update AST/TO ratio

In [7]:
game = GameData('Fairport')
game.data

Retrieving Fairport roster data.


,PTS,FGM,FGA,FG%,FT%,2P%,3P%,REB,HUSTLE,2PM,2PA,3PM,3PA,FTM,FTA,DREB,OREB,AST,STL,DFL,BLK,TOV,PFL,CHARGE,PENNY,eFG%,TS%,AST/TO,PACE,OFFRTG,DEFRTG,NETRTG,USG%,AST Ratio,OREB%,DREB%,PAINT TOUCH
Player ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#2 Bruce Wilder,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#3 Derek Howe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#5 Garrett Kucera,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#10 Tyler Pucci,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#11 Alex Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#12 Josh Knapp,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#13 Andrew Smith,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#15 James Stanek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#20 Ian Kennedy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Helper function to split tooltip
def parse_tooltip(button):
    name, stat = button.tooltip.split(",")
    return name[1:], stat[:-1]

# Add functionality to automatically ++ attempts when there's a make
def handle_submit(sender):
    name, stat = parse_tooltip(sender)
    game.stats_update_dict[stat](name, stat)
    sender.description = game.data.loc[name,stat].astype(int).astype(str)
    
box_layout = Layout(border='solid', width='1200px') # Define consistent box layout style
button_layout = Layout(height='auto', width='auto') # Define consistent button layout style

player_ids = []; boxes = []; stats = [] #icons = icons()

p_id = "Player ID"; p_id = p_id.center(game.n, "_") # Center str for proper formatting
id_button = Button(description=p_id, disabled=True, button_style='success', layout=button_layout)
stats.append(id_button)

for i, player_id in enumerate(game.data.index):
    t = Button(description=player_id, disabled=True, button_style='danger', layout=button_layout)
    player_ids.append(t)
    buttons = []
    for j, stat in enumerate(game.stats['entry']):
        if i == 0:
            stats.append(Button(description=stat, disabled=True, button_style='success', layout=button_layout))
        tool_tip = "(" + player_id + "," + stat + ")"
        wid = str(max(len(stat)*14, 45)) + "px"
        player_button = Button(description="0".center(len(stat)), disabled=False, tooltip=tool_tip,
                               layout=Layout(height='auto', width=wid))
        player_button.on_click(handle_submit)
        buttons.append(player_button)
    boxes.append(HBox(buttons))

data_entry_ui = VBox([HBox(stats, box_style='success', layout=box_layout), # stats_descript
                      HBox([VBox(player_ids), VBox(boxes)], box_style='warning', layout=box_layout)], # player_data
                     layout=box_layout)

display(data_entry_ui)

<ipython-input-3-d6cfec8c580f>:74: RuntimeWarning: divide by zero encountered in long_scalars
  self.data.loc[name,'AST/TO'] = self.data.loc[name,'AST']/self.data.loc[name,'TOV']


In [9]:
data_entry_ui.close()
game.data

,PTS,FGM,FGA,FG%,FT%,2P%,3P%,REB,HUSTLE,2PM,2PA,3PM,3PA,FTM,FTA,DREB,OREB,AST,STL,DFL,BLK,TOV,PFL,CHARGE,PENNY,eFG%,TS%,AST/TO,PACE,OFFRTG,DEFRTG,NETRTG,USG%,AST Ratio,OREB%,DREB%,PAINT TOUCH
Player ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#2 Bruce Wilder,2.0,1.0,3.0,0.333333,0.0,0.333333,0,0.0,0,1.0,3.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.333333,0.333333,0.0,0,0,0,0,0,0,0,0,0
#3 Derek Howe,2.0,1.0,3.0,0.333333,0.0,0.333333,0,0.0,0,1.0,3.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.333333,0.333333,0.0,0,0,0,0,0,0,0,0,0
#5 Garrett Kucera,1.0,0.0,0.0,0.000000,0.5,0.000000,0,1.0,0,0.0,0.0,0,0,1.0,2.0,0.0,1.0,0,0,0,0,0,0,0,0,0.000000,0.568182,0.0,0,0,0,0,0,0,0,0,0
#10 Tyler Pucci,1.0,0.0,0.0,0.000000,1.0,0.000000,0,2.0,0,0.0,0.0,0,0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0.000000,1.136364,0.0,0,0,0,0,0,0,0,0,0
#11 Alex Hill,0.0,0.0,0.0,0.000000,0.0,0.000000,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0
#12 Josh Knapp,0.0,0.0,0.0,0.000000,0.0,0.000000,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0
#13 Andrew Smith,1.0,0.0,0.0,0.000000,1.0,0.000000,0,0.0,0,0.0,0.0,0,0,1.0,1.0,0.0,0.0,0,0,0,1,0,0,0,0,0.000000,1.136364,0.0,0,0,0,0,0,0,0,0,0
#15 James Stanek,1.0,0.0,0.0,0.000000,1.0,0.000000,0,0.0,0,0.0,0.0,0,0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0.000000,1.136364,0.0,0,0,0,0,0,0,0,0,0
#20 Ian Kennedy,0.0,0.0,0.0,0.000000,0.0,0.000000,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0,0,0,0,0,0,0,0,0
